# OLAP queries
To run these queries, make a [database.ini](./database.ini) file with content of this form:
```[postgresql]
host=localhost
database=firedb
user=[Enter Username here]
password=[Enter pass here]
```

In [5]:
from configparser import ConfigParser
import psycopg2
import pandas as pd
import pandas.io.sql as sqlio

def connect(config):
    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psycopg2.connect(**config) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)


def load_config(filename='database.ini', section='postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    # get section, default to postgresql
    config = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            config[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return config



In [7]:

config = load_config()
connection = connect(config)
assert connection
connection = connection

cursor = connection.cursor()

Connected to the PostgreSQL server.


## Rollup Query

Show amount of hectares burnt and the cost of those fires across years

In [9]:
rollup_query = """ SELECT EXTRACT(YEAR FROM burncostdate) AS year,SUM(hectaresburnt) \
    AS hectaresburnt,SUM(cost) AS cost FROM dailyburncost GROUP BY year ORDER BY year;"""


data = sqlio.read_sql_query(rollup_query,con=connection)
data





/tmp/ipykernel_71576/1880374246.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = sqlio.read_sql_query(rollup_query,con=connection)


,year,hectaresburnt,cost
0,1986.0,1.702788e+04,8.799761e+06
1,1987.0,3.592345e+04,1.003271e+07
2,1988.0,3.706151e+04,1.575455e+07
3,1989.0,1.378640e+06,3.276869e+08
4,1990.0,2.083241e+04,1.025886e+07
5,1991.0,8.521406e+04,3.629475e+07
6,1992.0,6.704579e+04,1.147374e+07
7,1993.0,6.894182e+04,7.926416e+06
8,1994.0,5.263515e+05,7.830021e+07
9,1995.0,8.666350e+05,2.867887e+08


## Drill down

Drill down the burn date by adding the distinction of province for each year

In [10]:

drilldown_query = """ SELECT EXTRACT(YEAR FROM burncostdate) AS year,fireprovinceshort AS province,SUM(hectaresburnt) \
    AS hectaresburnt,SUM(cost) AS cost FROM dailyburncost GROUP BY GROUPING SETS ((year,province)) ORDER BY year;"""


data = sqlio.read_sql_query(drilldown_query,con=connection)
data

/tmp/ipykernel_71576/3786639476.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = sqlio.read_sql_query(drilldown_query,con=connection)


,year,province,hectaresburnt,cost
0,1986.0,ON,16831.541085,8.773381e+06
1,1986.0,SK,196.340382,2.638001e+04
2,1987.0,ON,14227.927923,7.416257e+06
3,1987.0,SK,21695.517855,2.616457e+06
4,1988.0,ON,23462.859611,1.413849e+07
...,...,...,...,...
245,2022.0,QC,939.275741,3.454908e+06
246,2022.0,MB,15559.216099,1.965209e+07
247,2022.0,NL,1348.362417,4.437427e+06
248,2022.0,NS,54.008263,7.995234e+04


## Slice

An OLAP Slice operation we can do by only considering fires in 2022

In [18]:
slice_query = """ SELECT * FROM dailyburncost WHERE EXTRACT(YEAR FROM burncostdate)=2022 ORDER BY burncostdate"""


data = sqlio.read_sql_query(slice_query,con=connection)
data

/tmp/ipykernel_71576/2423623206.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = sqlio.read_sql_query(slice_query,con=connection)


,stationid,burncostdate,provinceid,burnincidentid,fireprovinceshort,averagetemperature,maxrelativehumidity,maxwindspeedgust,hectaresburnt,cost
0,3608,2022-02-13,8,257170,AB,-2.0,100.0,31.0,0.050305,71.152275
1,3375,2022-03-23,8,256865,AB,11.5,NaN,NaN,0.094233,133.283341
2,635,2022-04-23,9,252012,BC,9.9,96.0,NaN,0.049156,198.651495
3,3422,2022-04-24,8,251337,AB,6.2,94.0,37.0,0.886300,1253.589332
4,5690,2022-04-30,5,254631,ON,7.2,79.0,NaN,0.639585,4649.222251
...,...,...,...,...,...,...,...,...,...,...
5616,1128,2022-11-18,9,249370,BC,-3.8,79.0,37.0,0.307776,1243.809830
5617,1663,2022-11-18,9,252643,BC,-15.2,89.0,NaN,74.673126,301774.922711
5618,1663,2022-11-19,9,252644,BC,-10.9,91.0,NaN,74.673126,301774.922711
5619,1663,2022-11-20,9,252645,BC,-6.5,95.0,NaN,74.673126,301774.922711


## Dice
Dice queries tried we will select aggregates from only BC and QC to be able to highlight differences for these provinces on the two sides of Canada

Secondly we try grabbing only those fires which burnt more than 200 hectares in a day, or caused a cost equivalent to the east costly of the 200+ hectare fires.

In [21]:
dice1_query = """ SELECT EXTRACT(YEAR FROM burncostdate) AS year,fireprovinceshort AS province,SUM(hectaresburnt) \
    AS hectaresburnt,SUM(cost) AS cost FROM dailyburncost WHERE fireprovinceshort='QC' OR fireprovinceshort='BC' GROUP BY GROUPING SETS ((year,fireprovinceshort)) ORDER BY year;"""


data = sqlio.read_sql_query(dice1_query,con=connection)
data

/tmp/ipykernel_71576/1119042269.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = sqlio.read_sql_query(dice1_query,con=connection)


,year,province,hectaresburnt,cost
0,1989.0,BC,801.220308,3.119279e+05
1,1989.0,QC,417050.896751,1.318158e+08
2,1990.0,BC,1200.257824,5.260539e+05
3,1990.0,QC,1531.475724,5.323908e+05
4,1991.0,QC,35161.019538,1.306265e+07
...,...,...,...,...
63,2020.0,BC,1567.259261,4.724611e+06
64,2021.0,QC,13044.452811,4.261223e+07
65,2021.0,BC,386232.704092,1.377371e+09
66,2022.0,QC,939.275741,3.454908e+06


In [29]:
# First find smallest cost of 200 hectare fire

dice2_query = f""" SELECT * FROM dailyburncost WHERE hectaresburnt>200.0 OR cost > {min_cost} ORDER BY hectaresburnt ASC;"""


data = sqlio.read_sql_query(dice2_query,con=connection)
data

/tmp/ipykernel_71576/4246563249.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  min_data = sqlio.read_sql_query(min_query,con=connection)
/tmp/ipykernel_71576/4246563249.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = sqlio.read_sql_query(dice2_query,con=connection)


,stationid,burncostdate,provinceid,burnincidentid,fireprovinceshort,averagetemperature,maxrelativehumidity,maxwindspeedgust,hectaresburnt,cost
0,5407,2021-07-28,5,246454,ON,19.0,NaN,NaN,4.021554,2.349297e+04
1,5407,2021-07-07,5,246433,ON,16.3,NaN,NaN,4.021554,2.349297e+04
2,5407,2021-07-25,5,246451,ON,21.8,NaN,NaN,4.021554,2.349297e+04
3,5407,2021-07-24,5,246450,ON,21.3,NaN,NaN,4.021554,2.349297e+04
4,5407,2021-07-26,5,246452,ON,20.8,NaN,NaN,4.021554,2.349297e+04
...,...,...,...,...,...,...,...,...,...,...
41405,7357,2013-07-07,0,149153,NL,15.8,98.0,37.0,7180.876304,8.450825e+06
41406,7357,2013-07-06,0,149152,NL,18.8,100.0,52.0,7180.876304,8.450825e+06
41407,7357,2013-07-05,0,149151,NL,21.5,96.0,32.0,7180.876304,8.450825e+06
41408,7357,2013-07-04,0,149150,NL,16.5,92.0,32.0,7180.876304,8.450825e+06


## Combined OLAP queries

1. Get average cost and spread of fires per province considering only those fires at least 10 hectares large

2. Get average daily temperature, hectares burnt and cost for fires by year, also only considering fires 10 hectares large

3. Get fire incidents occuring in the month of August with average temperature available and list all in order of temperature

4. Get average of average daily temp, total burnt hectares and cost across burnincidents occuring in Ontario in June-August grouped by year. 

In [45]:
comb1_query = f""" SELECT fireprovinceshort, AVG(cost) as cost,AVG(hectaresburnt) as avg_burnt, COUNT(burnincidentid) as fires FROM dailyburncost WHERE hectaresburnt > 10 GROUP BY fireprovinceshort"""


data = sqlio.read_sql_query(comb1_query,con=connection)
data

/tmp/ipykernel_71576/2337658677.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = sqlio.read_sql_query(comb1_query,con=connection)


,fireprovinceshort,cost,avg_burnt,fires
0,NL,354559.550604,306.106587,571
1,NS,64389.882834,66.647303,10
2,QC,163011.241178,200.983088,6750
3,ON,253802.059060,93.875348,9739
4,MB,61366.153145,160.537982,14950
5,SK,51334.116664,154.959061,18304
6,AB,70249.510283,102.418833,6006
7,BC,302068.770847,111.059441,9818


In [43]:
comb2_query = f""" SELECT EXTRACT(YEAR FROM burncostdate) as year, AVG(averagetemperature) as avg_temp,AVG(hectaresburnt) as avg_burnt, SUM(cost) as total_cost FROM dailyburncost \
     WHERE hectaresburnt > 10 GROUP BY year ORDER BY year"""


data = sqlio.read_sql_query(comb2_query,con=connection)
data

/tmp/ipykernel_71576/3548797269.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = sqlio.read_sql_query(comb2_query,con=connection)


,year,avg_temp,avg_burnt,total_cost
0,1986.0,15.065054,84.474874,8.189995e+06
1,1987.0,16.138957,44.888210,8.684482e+06
2,1988.0,16.056715,34.954511,1.158350e+07
3,1989.0,17.912070,314.347920,3.238655e+08
4,1990.0,15.832741,45.429885,8.768304e+06
5,1991.0,17.467824,76.437730,3.557038e+07
6,1992.0,14.533806,155.781434,1.088983e+07
7,1993.0,12.699422,194.762020,7.470136e+06
8,1994.0,15.614836,184.433971,7.715547e+07
9,1995.0,16.480870,205.927988,2.838053e+08


In [54]:
comb3_query = f"""SELECT burncostdate,fireprovinceshort,averagetemperature,hectaresburnt,cost FROM dailyburncost \
    WHERE averagetemperature IS NOT NULL AND EXTRACT(MONTH FROM burncostdate)=8 ORDER BY averagetemperature DESC"""


data = sqlio.read_sql_query(comb3_query,con=connection)
data

/tmp/ipykernel_71576/3392888257.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = sqlio.read_sql_query(comb3_query,con=connection)


,burncostdate,fireprovinceshort,averagetemperature,hectaresburnt,cost
0,2004-08-15,BC,31.0,0.259440,262.065231
1,2004-08-15,BC,31.0,1.977424,1997.430053
2,2004-08-15,BC,31.0,5.551442,5607.608163
3,2004-08-15,BC,31.0,16.551772,16719.231957
4,2018-08-12,MB,30.4,0.007599,6.864033
...,...,...,...,...,...
47955,2001-08-21,ON,0.8,0.353630,433.344010
47956,1987-08-10,SK,0.5,0.660856,79.698562
47957,2018-08-28,QC,0.4,3.331455,8253.810606
47958,2006-08-21,ON,0.4,3.488514,5930.897679


In [62]:
comb4_query = f"""SELECT EXTRACT(YEAR FROM burncostdate) as year, AVG(averagetemperature) as avg_temp,SUM(hectaresburnt) as total_hectare_burnt,SUM(cost) as cost FROM \
    (SELECT * FROM dailyburncost WHERE EXTRACT(MONTH FROM burncostdate)=6 OR EXTRACT(MONTH FROM burncostdate)=7 OR EXTRACT(MONTH FROM burncostdate)=8 )\
    WHERE fireprovinceshort='ON' GROUP BY year ORDER BY year ASC"""


data = sqlio.read_sql_query(comb4_query,con=connection)
data

/tmp/ipykernel_71576/615863517.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = sqlio.read_sql_query(comb4_query,con=connection)


,year,avg_temp,total_hectare_burnt,cost
0,1986.0,14.469952,13564.149788,7.070265e+06
1,1987.0,16.545920,13227.671734,6.894877e+06
2,1988.0,18.000817,17475.794744,1.053074e+07
3,1989.0,18.220065,106844.983202,8.250110e+07
4,1990.0,16.400830,9200.854855,7.994430e+06
5,1991.0,17.587640,19651.120954,1.831507e+07
6,1992.0,12.824260,1637.981851,1.447735e+06
7,1993.0,15.806329,654.267586,5.676850e+05
8,1994.0,17.094249,1252.518865,1.081698e+06
9,1995.0,18.146011,137297.088666,1.215727e+08


## Iceberg Query

As an iceberg query we want to look at which weeks of the year have the greatest amount of fire incidents. 

In [36]:
iceberg_query = f""" SELECT EXTRACT(WEEK FROM burncostdate) as week, AVG(averagetemperature) avg_temp, COUNT(burnincidentid) \
    AS fires, AVG(cost) as avg_cost FROM dailyburncost GROUP BY week ORDER BY fires DESC LIMIT 10;"""


data = sqlio.read_sql_query(iceberg_query,con=connection)
data

/tmp/ipykernel_71576/945569989.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = sqlio.read_sql_query(iceberg_query,con=connection)


,week,avg_temp,fires,avg_cost
0,29.0,17.755404,14869,67915.836625
1,28.0,17.535135,14561,84718.622319
2,30.0,17.739053,14388,57222.219178
3,31.0,17.299358,14017,55380.379730
4,27.0,16.919981,13563,76063.626931
5,32.0,17.460192,13035,62704.451024
6,33.0,16.882128,11465,56814.513644
7,26.0,16.588465,11417,45434.346045
8,25.0,15.653365,9926,34526.378919
9,34.0,15.785640,8788,51673.873800


## Windowing

As an example of windowing below there is a ranking of months by most hectares burnt for each year

In [77]:
windowing_query = f"""SELECT EXTRACT(YEAR FROM burncostdate) as year, EXTRACT(MONTH FROM burncostdate) as month,AVG(averagetemperature) as avg_temp,SUM(hectaresburnt) as total_hectare_burnt,SUM(cost) as cost, \
    RANK() OVER (PARTITION BY EXTRACT(YEAR FROM burncostdate) ORDER BY SUM(hectaresburnt) DESC)\
    FROM dailyburncost GROUP BY GROUPING SETS ((year,month)) ORDER BY year,rank"""



data = sqlio.read_sql_query(windowing_query,con=connection)
data

/tmp/ipykernel_71576/1388665075.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = sqlio.read_sql_query(windowing_query,con=connection)


,year,month,avg_temp,total_hectare_burnt,cost,rank
0,1986.0,6.0,12.967687,9445.239986,4.890739e+06,1
1,1986.0,7.0,16.335377,3971.009141,2.065723e+06,2
2,1986.0,5.0,16.169620,3185.504796,1.653528e+06,3
3,1986.0,8.0,15.026203,304.044278,1.347822e+05,4
4,1986.0,9.0,7.692754,87.116181,4.540900e+04,5
...,...,...,...,...,...,...
296,2022.0,11.0,-5.046154,1573.675615,6.359662e+06,6
297,2022.0,5.0,11.275610,141.360946,4.149999e+05,7
298,2022.0,4.0,7.766667,1.575041,6.101463e+03,8
299,2022.0,3.0,11.500000,0.094233,1.332833e+02,9


## Window Clause

As an example use of the window clause, there is below a calculation of the percentage of hectares burnt in a 3 month moving average that a burnincident contributes to.

In [154]:
windowing_query = f"""SELECT burnincidentid,EXTRACT(YEAR FROM burncostdate) as year,EXTRACT(MONTH FROM burncostdate) as month,hectaresburnt, hectaresburnt / SUM(hectaresburnt) OVER W AS fires_mov_avg \
FROM dailyburncost GROUP BY GROUPING SETS ((burnincidentid,year,month,hectaresburnt))\
WINDOW W AS (PARTITION BY EXTRACT(YEAR FROM burncostdate)\
ORDER BY EXTRACT(MONTH FROM burncostdate)\
RANGE BETWEEN '1' PRECEDING \
AND '1' FOLLOWING)"""


data = sqlio.read_sql_query(windowing_query,con=connection)
data

/tmp/ipykernel_71576/4207456019.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = sqlio.read_sql_query(windowing_query,con=connection)


,burnincidentid,year,month,hectaresburnt,fires_mov_avg
0,188,1986.0,4.0,4.469712,0.001393
1,189,1986.0,4.0,4.469712,0.001393
2,190,1986.0,4.0,4.469712,0.001393
3,186,1986.0,4.0,4.469712,0.001393
4,187,1986.0,4.0,4.469712,0.001393
...,...,...,...,...,...
179112,252638,2022.0,11.0,74.673126,0.006994
179113,252646,2022.0,11.0,74.673126,0.006994
179114,252645,2022.0,11.0,74.673126,0.006994
179115,252644,2022.0,11.0,74.673126,0.006994
